<a href="https://colab.research.google.com/github/mkhetan/ph2_session9/blob/master/p2_session9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet envs
import gym
import torch
import torch.nn as nn
import torch.nn.fuctional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque


SyntaxError: ignored

In [0]:
class ReplayBuffer(object):
  def __init__(self, maxsize = 1e6):
    self.storage[]
    self.max_size = max_size
    self.ptr = 0

def add(self, transition):
  if len(self.storage) == self.max_size:
    self.storage[int(self.ptr)] = tranistion
    self.ptr = (self.ptr + 1) % self.max_size
  else:
    self.storage.append(transition)

def sample(self, batch_size):
  ind = np.random.randint(0, len(self.storage), batch_size)
  batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [],[],[],[],[]
  for i in ind:
    state, next_state, action, reward, done = self.storage[i]
    batch_states.append(np.array(state, copy = False))
    batch_next_states.append(np.array(next_state, copy = False))
    batch_actions.append(np.array(action, copy = False))
    batch_rewards.append(np.array(reward, copy = False))
    batch_dones.append(np.array(done, copy = False))
  return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1,-1), np.array(batch_dones).reshape(-1,-1)


Actor network - predicts "actions" which are nothing but Q values and the action we take is the one that has the max Q value... but let us say - Actor gives us "actions"

In [0]:
class Actor(nn.Module):

  def __init__(self, state_dims, action_dim, max_action):
    #max_action is to clip in case we added too much noise
    super(Actor, self).__init__() # activate the inheritence
    self.layer_1 = nn.Linear(state_dims, 400)
    self.layer_2 = nn.Linear(400,300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.action_max = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action*torch.tanh(self.layer_3(x))
    return x



Critic network. There is same definition for critic1 and critic2
There is no need to clip actions. This network receives clipped actions only from Actor
state_dims - input dimensions... e.g. cartpole input was a patch of image. here it could be many moving parts of a robot for example.

In [0]:
class Critic(nn.Module):

  def __init__(self, state_dims, action_dim):
    super(Critic, self).__init__() # activate the inheritence
    # First critic network
    self.layer_1 = nn.Linear(state_dims + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    # Second Critic network - we are defining it here seperately because it is going to have different
    # waights compared to the first critic
    self.layer_4 = nn.Linear(state_dims + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, action_dim)

  def forward(self, x, u): # x - state, u - action
    xu = torch.cat([x, u], 1) # 1 for vertical concat and 0 for horizontal
    # forward propagation on first critic
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)

    # forward propagation on second Critic
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)

    return x1, x2

# The following is defined separately because we want to train our actor later on
# There will be no back prop on this. That is why it is kept separately
  def Q1(self, x, u): # x - state, u = action This is used for updating the Q values
    xu = torch.cat([x, u], 1) # 1 for vertical concat and 0 for horizontal
    # We decided to use the first critic to train our Actor
    # Any of them could have been used - or even avg of 1st and 2nd... but we
    # are using the 1st
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1


Training Process - Steps 4 to 15

In [0]:
# Selecting the device CPU or GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Building the whole training processs into a class

class T3D(object):
  def __init__(self, state_dims, action_dim, max_action):

    #making sure our T3D class can work with any env
    self.actor = Actor(state_dims, action_dim, max_action).to(device) #GD This is actor model where we are going to run back prop
    self.actor_target = Actor(state_dims, action_dim, max_action).to(device) # Polyak avg
    self.actor_target.load_state_dict(self, self.actor.state_dict) # during init - the wigths of actor model and actor target are same
    # initializing with model weights to keep them same
    # Adam optimizer - reinforcement learning uses this instead of SGD
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

    self.critic = Critic(state_dims, action_dim).to(device) # GD
    self.critic_target = Critic(state_dims, action_dim).to(device) # Polyak Avg
    self.critic_target.load_state_dict(self, critic.state_dict) # during init - the wigths of critic model and critic target are same

    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  # we need actions to train our critic
  def select_action(self, state):

    state = torch.Tensor(state.reshape(1,-1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()
    # need to convert to numpy, remeber clipping?

  # iterations - how many iterations you want to run the training loop for, discount is gamma, tau is for polyak avg, policy noise is noise we add to our actions
  # noise_clip - this is our max action amplitude for our env (moving only in max -.5 quanta)
  # policy_freq - how often are we going to update our actor 
  def train(self, replay_buffer, iterations, batch_size=100, discount = 0.99, tau = 0.005, policy_noise = 0.2, noise_clip = 0.5, policy_freq = 2):
    for it in range(iterations):
      # step 4 is taking a sample from a batch of transitions (s,s',a,r) from memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      # Each of the above need to be sent to our GPU / CPU
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      # Step 5 : from the next state s', the actor target plays the next action a'
      # mayank - doubt - but this is a list of actions - not "the" action - isn't it?
      next_action = self.actor_target.forward(next_state)

      #step 6 : add gausian noise to a' and clamp it to a value that is allowed by our env
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      # So our next_action is next_action from the network + noise - then clamp it by
      # limitations of the environment (self.max_action)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

      # step 7 : this next action a' and next state s' is fed to the critics
      # Qt1(s',a') and Qt2(s',a')
      target_Q1, target_Q2 = self.critic_target.forward(next_state, next_action)

      # step 8: take the min of the 2
      target_Q = torch.min(target_Q1, target_Q2)

      # Step 9.. so the actual target_Q = reward + gamma*target_Q <--- this target_Q was obtained above
      # but we should not consider the steps where done - 1 
      target_Q = reward + ((1 - done)*gamma*target_Q).detach # this detach is needed for a reason i do not understand

      # step 10: The 2 critic "models" take (s,a) and give out Q1 and Q2
      current_Q1, current_Q2 = self.critic.forward(self, action)

      # step 11: We calculate the loss from the 2 critic models
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

      # step 12: Back prop on critic models using the above loss
      self.critic_optimizer.zero_grad() # initializing the gradients to 0
      critic_loss.backward() # computing the gradient
      self.critic_optimizer.step() # performing the weight updates

      # step 13:
      # Once every 2 iterations, we update the Actor model by performing gradient ascent
      # on the o/p of the "first" critic model (we chose it - it could have been 2nd or avg of both)

      if it % policy_freq == 0:
        # This is DPG part?
        # Basically - we run forward propagation on critic model 1 (Q1) - feed it the current state and
        # output of actor when the same state was passed... calculate mean of the critic outputs
        # But what is this loss compared to? it should be compared to how good this could have been? How 
        # will we know that?
        actor_loss = -(self.critic.Q1(state, self.actor(state)).mean())
        self.actor_optimizer.grad_zero()
        actor_loss.backward()
        self.actor_optimizer.step()

      # Step 14: Every 2 iterations, perform polyak averaging on actor target from actor model
      # remember - tau is .005 - so, it is taking a very small weight of actor model
      for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
        target_param.data.copy_(tau * param.data + (1-tau)*target_param.data)

      # step 15: Once every 2 iterations, we update the critic target the same way
      for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
        target_param.data.copy_(tau * param.data + (1-tau)*target_param.data)
